# Regression Concep Vectors for Bidirectional Relevance Scores in Histopathology

In [3]:
import sys
sys.path.append('./keras_vis_tcav/')
import tcav_utils
import os
import numpy as np
import functions
import models
import keras.backend as K
reload(tcav_utils)

import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import scipy.stats
import keras.backend as K
import numpy as np
import sklearn.model_selection
import sklearn.linear_model
import h5py

As a first thing, we load ResNet101 finetuned to classify between tumor and non-tumor patches. 
The network has been trained on 224x224 patches randomly sampled from the highest resolution level of the WSIs in Camelyon16 (hopefully) and Camelyon17. 

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"]="0" 
CONFIG_FILE='./models/0528-1559/config.cfg'
settings = functions.parseTrainingOptions(CONFIG_FILE)
bc_model = models.getModel(settings)
bc_model.load_weights('./models/0528-1559/tumor_classifier.h5')

./camnet/resnet101.py:73: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (1, 1), strides=(1, 1), use_bias=False, name="res2a_branch2a")`
  name=conv_name_base + '2a', bias=False)(input_tensor)
./camnet/resnet101.py:80: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), use_bias=False, name="res2a_branch2b")`
  name=conv_name_base + '2b', bias=False)(x)
./camnet/resnet101.py:85: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (1, 1), use_bias=False, name="res2a_branch2c")`
  x = Convolution2D(nb_filter3, 1, 1, name=conv_name_base + '2c', bias=False)(x)
./camnet/resnet101.py:90: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (1, 1), strides=(1, 1), use_bias=False, name="res2a_branch1")`
  name=conv_name_base + '1', bias=False)(input_tensor)
./camnet/resnet101.py:94: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.mer

./camnet/resnet101.py:47: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1024, (1, 1), use_bias=False, name="res4b3_branch2c")`
  x = Convolution2D(nb_filter3, 1, 1, name=conv_name_base + '2c', bias=False)(x)
./camnet/resnet101.py:35: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (1, 1), use_bias=False, name="res4b4_branch2a")`
  x = Convolution2D(nb_filter1, 1, 1, name=conv_name_base + '2a', bias=False)(input_tensor)
./camnet/resnet101.py:42: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), use_bias=False, name="res4b4_branch2b")`
  name=conv_name_base + '2b', bias=False)(x)
./camnet/resnet101.py:47: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1024, (1, 1), use_bias=False, name="res4b4_branch2c")`
  x = Convolution2D(nb_filter3, 1, 1, name=conv_name_base + '2c', bias=False)(x)
./camnet/resnet101.py:35: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (1, 1), use_b

./camnet/resnet101.py:47: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1024, (1, 1), use_bias=False, name="res4b16_branch2c")`
  x = Convolution2D(nb_filter3, 1, 1, name=conv_name_base + '2c', bias=False)(x)
./camnet/resnet101.py:35: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (1, 1), use_bias=False, name="res4b17_branch2a")`
  x = Convolution2D(nb_filter1, 1, 1, name=conv_name_base + '2a', bias=False)(input_tensor)
./camnet/resnet101.py:42: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), use_bias=False, name="res4b17_branch2b")`
  name=conv_name_base + '2b', bias=False)(x)
./camnet/resnet101.py:47: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1024, (1, 1), use_bias=False, name="res4b17_branch2c")`
  x = Convolution2D(nb_filter3, 1, 1, name=conv_name_base + '2c', bias=False)(x)
./camnet/resnet101.py:35: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (1, 1), u

We load the dataset used to extract concepts about the nuclei morphology and texture. The DATASETNAME for nuclei segmentation contains 6 Breast Cancer WSIs. From them we sample 300 random patches and we compute statistics on the nuclei morphology and texture. 

In [ ]:
patches, masks, nuclei, stats = tcav_utils.get_norm_patches(path='./datasets/training/'+str(0))

In [ ]:
# Nuclei morphology statistics
nuclei_morph = tcav_utils.nuclei_morphology(stats)
# Nuclei texture statistics
nuclei_text = tcav_utils.nuclei_texture(patches, nuclei)

We predict the cancer probability for each one of the concept-patches

In [ ]:
inputs = np.float64(patches)
inputs = models.standardPreprocess(inputs)
predictions = bc_model.predict(inputs[:50])

In [ ]:
predictions = predictions.reshape((50,))
os.mkdir('results/')
np.save('results/predictions', predictions)

## Step 0. Correlation Analysis


We first show the correlation between some characteristics of the nuclei and the network predictions.  

In [ ]:
# Correlation analysis and p-values
def corr_analysis(feature, pred):
    return scipy.stats.pearsonr(np.array(feature), np.array(pred))
# Correlation analysis of morphology statistics
print 'area: ', corr_analysis(np.array(nuclei_morph['area'][:50]).reshape((50,)), predictions)
print 'perimeter: ', corr_analysis(nuclei_morph['perimeter'][:50], predictions)
print 'eccentricity: ', corr_analysis(nuclei_morph['eccentricity'][:50], predictions)
print 'mjaxis: ', corr_analysis(nuclei_morph['mjaxis'][:50], predictions)
print 'euler: ', corr_analysis(nuclei_morph['euler'][:50], predictions)
# Correlation analysis of texture features
print 'contrast: ', corr_analysis(nuclei_text['contrast'][:50], predictions)
print 'ASM: ', corr_analysis(nuclei_text['ASM'][:50], predictions)
print 'correlation: ', corr_analysis(nuclei_text['correlation'][:50], predictions)

Contrast and dissimilarity seem to be correlated with the predictions, as well. We will analyse these concepts.
NB. Correlation seems negatively correlated with the prediction, so we could potentially think of it as a Reverse Concept for classification, which is indicative of the class non-cancer. 

Extracting ResNet features for concepts. 
We will now extract high-dimensional feature representations of the patches with a forward pass. 
These features will then be used to learn the concept vectors. 
We analyse three different levels in the network.

# Step 2. Linear Regression in the activation space

In [3]:
layers = ['conv1',
          'res2a',
          'res2b',
          'res2c',
          'res3a', # quite early in the network
          'res3b1',
          'res3b2',
          'res3b3',
          'res4a', # middle
          'res4b1',
          'res4b2',
          'res4b3',
          'res4b4',
          'res4b5',
          'res4b6',
          'res4b7',
          'res4b8',
          'res4b9',
          'res4b10',
          'res4b15',
          'res4b16',
          'res4b17',
          'res4b18',
          'res4b19',
          'res4b20',
          'res5a'  # closer to the last layers
         ]
l = 'res4a'
concepts=['perimeter',
 'area',
 'mjaxis',
 'eccentricity',
 'euler',
 'contrast', 
 'ASM',
 'correlation',]
max_rep = 30

In [ ]:
for repetition in range(0, max_rep):
    tr_set=tcav_utils.get_cv_training_set('./datasets/breast_nuclei/Tissue Images/', repetition)
    patches, masks, nuclei, stats = tcav_utils.get_norm_patches(path='./datasets/training/'+str(repetition))
    nuclei_morph = tcav_utils.nuclei_morphology(stats)
    # Nuclei texture statistics
    nuclei_text = tcav_utils.nuclei_texture(patches, nuclei)
    inputs = np.float64(patches)
    inputs = models.standardPreprocess(inputs)
    get_layer_output = K.function([bc_model.layers[0].input],
                              [bc_model.get_layer(l).output])
    feats = get_layer_output([inputs])
    np.save('./rcv/phis/'+str(repetition)+'_concepts_phis_'+l, np.asarray(feats[0]))

In [ ]:
def solve_regression(inputs, y, n_splits=2, n_repeats=1, random_state=12883823, verbose=0):
    scores=[]
    max_score = 0
    direction = None
    rkf = sklearn.model_selection.RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=random_state)
    counter = 0
    for train, test in rkf.split(inputs):
        if verbose:
            print 'N. ', counter, '..'
        reg = sklearn.linear_model.LinearRegression()
        reg.fit(inputs[train], y[train])
        trial_score = reg.score(inputs[test], y[test])
        scores.append(trial_score)
        if trial_score > max_score:
            direction = reg.coef_
        if verbose:
            print trial_score
        counter += 1
    if verbose:
        print np.mean(scores)
    return np.mean(scores), direction

In [ ]:
feats=[]
for repetition in range(max_rep):
    patches, masks, nuclei, stats = tcav_utils.get_norm_patches(path='./datasets/training/'+str(repetition))
    nuclei_morph = tcav_utils.nuclei_morphology(stats)
    nuclei_text = tcav_utils.nuclei_texture(patches, nuclei)
    feats=np.load('./rcv/phis/'+str(repetition)+'_concepts_phis_res4a.npy')
    X=(np.asarray([x.ravel() for x in feats], dtype=np.float64))
    for c in concepts[:-3]:
        reg_score, cv = solve_regression(X, np.asarray(nuclei_morph[c]))
        np.save('./rcv/reg_score_'+c+'_'+str(repetition)+'.npy', reg_score)
        np.save('./rcv/rcv_'+c+'_'+str(repetition)+'.npy', cv)
    for c in concepts[-3:]:
        reg_score, cv = solve_regression(X, np.asarray(nuclei_text[c]))
        np.save('./rcv/reg_score_'+c+'_'+str(repetition)+'.npy', reg_score)
        np.save('./rcv/rcv_'+c+'_'+str(repetition)+'.npy', cv)
        

# Step 3. Sensitivity scores

We compute sensitivity scores as the directional derivative of the test inputs on the RCV in the activation space.

In [4]:
# loading the test inputs
PWD = '/mnt/nas2/results/IntermediateResults/Camelyon/all500'
h5file = 'patches.hdf5'
db = h5py.File(os.path.join(PWD, h5file), 'r')
os.path.join(PWD, h5file)
def print_info(name, obj):
    print name 
db.visititems(print_info)
tumor_patches = db['all_tumor_patches']
normalizer = tcav_utils.get_normalizer()
normalised_tumor_patches = np.array([tcav_utils.normalize_patch(np.uint8(patch), normalizer) for patch in tumor_patches[0:3000:10]])
test_inputs = np.float64(normalised_tumor_patches)
test_inputs = models.standardPreprocess(test_inputs)

all_normal_patches
all_tumor_patches
normal
normal/level7
normal/level7/centre0
normal/level7/centre0/patient004
normal/level7/centre0/patient004/node4
normal/level7/centre0/patient004/node4/locations
normal/level7/centre0/patient004/node4/patches
normal/level7/centre0/patient009
normal/level7/centre0/patient009/node1
normal/level7/centre0/patient009/node1/locations
normal/level7/centre0/patient009/node1/patches
normal/level7/centre0/patient010
normal/level7/centre0/patient010/node4
normal/level7/centre0/patient010/node4/locations
normal/level7/centre0/patient010/node4/patches
normal/level7/centre0/patient012
normal/level7/centre0/patient012/node0
normal/level7/centre0/patient012/node0/locations
normal/level7/centre0/patient012/node0/patches
normal/level7/centre0/patient015
normal/level7/centre0/patient015/node1
normal/level7/centre0/patient015/node1/locations
normal/level7/centre0/patient015/node1/patches
normal/level7/centre0/patient015/node2
normal/level7/centre0/patient015/node2/lo

[models] Appling some standard preprocessing to the data. 
[models] data mean:  54.57981152122549
[models] data std:  54.99523730633338


In [ ]:
for concept in concepts[5:]:
    for i in range(0, 30):
        print concept
        rcv = np.load('./rcv/rcv_'+concept+'_'+str(i)+'.npy')
        rcv /= np.linalg.norm(rcv)
        repetition = 0
        for p in range(len(test_inputs[:50])):
            nnn = tcav_utils.compute_tcav(bc_model,-1,0, np.expand_dims(test_inputs[p], axis=0), wrt_tensor=bc_model.get_layer('res4a').output)
            flattened_derivative=nnn.ravel()
            score = np.multiply(-1, np.dot(flattened_derivative, rcv))
            filet=open('tcav_'+concept+'_'+str(i)+'.txt', 'a')
            filet.write(str(repetition)+','+str(score)+'\n')
            filet.close()            

contrast
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?,

wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 

wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 

wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 

wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
contrast
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?,

...

wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 

wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 

wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
ASM
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 

wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 

wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 1024), dtype=float32)
wrt_tensor Tensor("res4a/add:0", shape=(?, 14, 14, 

We compute TCAV and Br scores and check statistical relevance

In [ ]:
def plot_scores(scores, legend, legend_entry, color):
    mu = np.mean(scores)
    variance = np.std(scores)
    sigma = math.sqrt(variance)

    x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
    plt.plot(x,mlab.normpdf(x, mu, sigma), color=color)
    plt.scatter([mu,mu,mu,mu], [0,0.25,0.5,0.75],marker='*',c=color, s=3)
    legend.append(legend_entry)
    return mu, variance, sigma, legend

In [ ]:
print 'Random TCAVs'
#print 0.06
N=50.0
mmax=30
TCAVs=np.zeros((mmax,))
Brs=np.zeros((mmax,))
VBrs=np.zeros((mmax,))
real_TCAVs=np.ones((mmax,))
real_Brs=np.zeros((mmax,))

#['area', 'contrast','eccentricity','ASM', 'correlation','euler']
for concept in concepts:
    TCAVs=np.zeros((mmax,))
    for i in range(0,10):
        df=[]
        df=pd.read_csv('./'+'tcavrandom_'+str(i)+'.txt', header=None)
        TCAV = np.sum(np.sign(np.array(df[1]))>0) / np.float(len(np.array(df[1])))
        R = 0
        Br = R * np.mean(np.array(df[1])) / np.std(np.array(df[1]))
        VBr = R * np.mean(np.array(df[1]))
        TCAVs[i]=TCAV
        Brs[i]=Br
        VBrs[i]=VBr
        #print TCAV
    for i in range(0,mmax):
        if os.path.exists('./'+'tcav_'+concept+'_'+str(i)+'.txt'):
            df=[]
            df=pd.read_csv('./'+'tcav_'+concept+'_'+str(i)+'.txt', header=None)
            #print './'+'tcavrandom_'+str(i)+'.txt'
            TCAV = np.sum(np.sign(np.array(df[1]))>0) /np.float(len(np.array(df[1])))
            R = np.load('./rcv/reg_score_'+concept+'_'+str(i)+'.npy')
            Br = R * np.mean(np.array(df[1])) / np.std(np.array(df[1]))
            VBr = R * np.mean(np.array(df[1]))
            real_TCAVs[i]=TCAV
            real_Brs[i]=Br
            real_VBrs[i]=VBr
                
    plt.figure() 
    leg =[]
    print TCAVs

    random_mu, random_variance, random_sigma, leg =  plot_scores(TCAVs, leg, 'random_TCAV', 'orange')
    mu, variance, sigma, leg =  plot_scores(real_TCAVs, leg, 'TCAV', 'green')
    br_mu, br_variange, br_sigma, leg =  plot_scores(real_Brs, leg, 'Br', 'red')

    plt.legend(leg[:4])
    plt.title(concept)
    plt.show()
    
    print 'random_TCAV', TCAVs, random_mu, random_variance, random_sigma
    print 'TCAV', real_TCAVs, mu, variance, sigma  
    
    print 'T-test" ', (mu - .5) * np.math.sqrt(len(real_TCAVs))/ (variance)

    print 'Br', br_mu, br_variange, br_sigma
    
    print 'Br T-test" ', (br_mu - 0) * np.math.sqrt(len(real_Brs))/ (br_variange)
    
    #print 'VBr', vbr_mu, vbr_variange, vbr_sigma
    
    #print 'TCAV', real_TCAVs, tmu, tsigma 
    #print 'Br', real_Brs, rmu, rsigma
    #print 'VBr', real_VBrs, mu, sigma